In [1]:
import sys
import csv
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import math

In [21]:
batch_size = 16
learning_rate = 1e-3
epochs = 25
validation_size = 50

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
    
print(DEVICE)

cuda


In [13]:
def clean_data(data, test=False):
    #delete survival
    if not test:
        data = np.delete(data, 1, 1)
    
    # delete names
    data = np.delete(data, 2, 1)
    
    # delete ticket number
    data = np.delete(data, 6, 1)
    
    # delete cabin #
    data = np.delete(data, 7, 1)
    
    # encode gender
    data[data=='male'] = 0.0
    data[data=='female'] = 1.0

    # encode port
    data[data=='C'] = 0.0
    data[data=='Q'] = 1.0
    data[data=='S'] = 2.0

    data[data==''] = -1
    
    data = data.astype(np.float)

    ids = data[:,0].astype(int)

    #delete id
    data = np.delete(data, 0, 1)

    data = list(zip(ids, data))
    
    return data

In [14]:
train_file = open('train.csv')
train_reader = csv.reader(train_file, delimiter=',')
train_data = np.array(list(train_reader)[1:])

test_file = open('test.csv')
test_reader = csv.reader(test_file, delimiter=',')
test_data = np.array(list(test_reader)[1:])

test_data = clean_data(test_data, True)

labels = list(zip(train_data[:,0].astype(int),train_data[:,1].astype(np.float)))

labels_map = {key: value for (key, value) in labels}

train_data = clean_data(train_data)

# class, name, gender, age, #siblings/spouses, #parents/children, ticket #, fare, cabin #, port

In [15]:
# shuffle and split for validation
np.random.shuffle(train_data)
validation_data = train_data[:validation_size]
train_data = train_data[validation_size:]

In [16]:
class FCModel(nn.Module):
    def __init__(self, input_dims):
        super().__init__()
        
        self.input_dims = input_dims

        self.fc1 = nn.Linear(7, 20000)
        self.fc2 = nn.Linear(self.fc1.out_features, 1000)
        self.fc3 = nn.Linear(self.fc2.out_features, 500)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
        
    def forward(self, input):
        x = F.relu(self.fc1(input))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [22]:
model = FCModel(7)

model = model.to(DEVICE)

step = 0

num_batches = math.ceil(len(train_data) / batch_size)

optimizer = torch.optim.Adam(model.parameters(), learning_rate)

criterion = nn.BCEWithLogitsLoss()

train_accuracies = []
validation_accuracies = []

def binary_accuracy(outputs, labels):
    return sum([1 if x == y else 0 for x,y in zip(torch.round(torch.sigmoid(outputs)), labels)]) / len(labels)

for i in range(epochs):
    #train
    model.train()
    
    for j in range(num_batches):
        start_index = j * batch_size
        end_index = start_index + batch_size if len(train_data) > batch_size * j + batch_size else len(train_data)
        
        batch = train_data[start_index:end_index]
        
        batch_labels = torch.Tensor([labels_map[x] for x,_ in batch]).unsqueeze(1)
        
        batch_labels = batch_labels
        
        batch_data = torch.Tensor([data for _,data in batch]).float()

        output = model.forward(batch_data)
        
        loss = criterion(output, batch_labels)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        accuracy = binary_accuracy(output, batch_labels)
        
        train_accuracies.append([step, accuracy])
        
        step += 1
        
        #print(f'epoch: ({i}/{epochs}) | step: ({step % num_batches}/{num_batches}) | loss: {loss:.5f} | accuracy: {100 * accuracy:.2f}')
        
    if validation_size > 0:
        #validate
        model.eval()
        with torch.no_grad():
            validation_labels = torch.Tensor([labels_map[x] for x,_ in validation_data]).unsqueeze(1)

            validation_input = torch.Tensor([data for _,data in validation_data]).float()

            output = model.forward(validation_input)

            accuracy = binary_accuracy(output, validation_labels)

            validation_accuracies.append([i, accuracy])

            print(f'validation | epoch: ({i}/{epochs}) accuracy: {100 * accuracy:.2f}')
        
plt.figure(1)
plt.ylim(0.0,1.0)

train_accuracies = np.array(train_accuracies)

plt.subplot(211)
plt.ylim(0.0,1.0)
plt.plot(train_accuracies[:,0], train_accuracies[:,1])

validation_accuracies = np.array(validation_accuracies)

plt.subplot(212)
plt.ylim(0.0,1.0)
plt.plot(validation_accuracies[:,0], validation_accuracies[:,1])

RuntimeError: CUDA error: an illegal memory access was encountered

In [124]:
#test
model.eval()

with torch.no_grad():
    test_input = torch.Tensor([data for _,data in test_data]).float()

    test_output = model.forward(test_input)

    test_output = torch.round(torch.sigmoid(test_output))

    predictions = list(zip([index for index,_ in test_data],test_output.numpy().astype(int)[:,0]))
    
    np.savetxt('submission.csv', [('PassengerId','Survived')] + predictions, delimiter=',', fmt='%s')
